<a href="https://colab.research.google.com/github/Silmar123/News_project/blob/main/meneame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importar librerías

import requests
import urllib 
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as bs

In [ ]:
#Lectura web
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:80.0) Gecko/20100101 Firefox/80.0'}
page = requests.get(url, headers=headers)

contenido = bs(page.content)

#Fecha en la que se envía la noticia

In [ ]:
def envio_de_noticia(contenido):

  try:
    hora_unix = int(contenido.find("span", attrs ={"title": "enviado: "} )["data-ts"])

    hora_madrid_delta = 2

    anadir_hora = timedelta(hours = hora_madrid_delta)

    fecha_de_envio = datetime.utcfromtimestamp(hora_unix) + anadir_hora

    fecha_de_envio = fecha_de_envio.strftime('%Y-%m-%d %H:%M:%S')

  except:

    fecha_de_envio = "Error fecha envío"

  return fecha_de_envio

#Fecha en la que se publica la noticia.

In [ ]:
def publicado_noticia(contenido):

  try:

    hora_unix = int(contenido.find("span", attrs ={"title": "publicado: "} )["data-ts"])

    hora_madrid_delta = 2

    anadir_hora = timedelta(hours = hora_madrid_delta)

    fecha_publicado = datetime.utcfromtimestamp(hora_unix) + anadir_hora

    fecha_publicado = fecha_publicado.strftime('%Y-%m-%d %H:%M:%S')

  except:

    fecha_publicado = "Error en fecha publicado"

  return fecha_publicado

#Función para extraer la información principal de cada noticia, de toda una página.

Devuelve una lista que contiene listas, con la información de cada noticia

In [ ]:
def info_noticia(contenido):
  
  info_noticia = []

  try:
    codigo_noticia = int(contenido.find("div")["data-link-id"])
  except AttributeError:
    codigo_noticia = "?"
  
  try:
    titular = contenido.find("a", attrs = {"class":"l:"+str(codigo_noticia)}).string
  except AttributeError:
    titular = "?"
  
  try:
    enlace_original = contenido.find("a", attrs = {"class":"l:"+str(codigo_noticia)})["href"]
  except AttributeError:
    enlace_original = "?"
  
  try:
    web_original = contenido.find("span", attrs = {"class":"showmytitle"}).string
  except AttributeError:
    web_original = "?"
  
  try:
    user_id=contenido.find("div", attrs={"class" : "news-submitted"}).a["class"][1]
    user_id_numerica=user_id[2:]
  except AttributeError:
    user_id="?"
    user_id_numerica="?"
  
  try:
    user_nick = contenido.find("div", attrs={"class" : "news-submitted"}).a["href"][6:]
  except AttributeError:
    user_nick = "?"
  
  try:
    etiqueta_principal = contenido.find("a", attrs={"class":"subname"}).string
  except AttributeError:
    etiqueta_principal = "?"
  
  try:
    numero_de_clics = contenido.find("div", attrs={"class":"clics"}).span.string
  except AttributeError:
    numero_de_clics="?"
  
  try:
    votos_positivos = contenido.find("span", attrs =  {"class":"positive-vote-number"}).string
  except AttributeError:
    votos_positivos = "?"
  
  try:
    votos_anonimos = contenido.find("span", attrs = {"class":"anonymous-vote-number"}).string
  except AttributeError:
    votos_anonimos = "?"
  
  try:
    votos_negativos = contenido.find("span", attrs = {"class":"negative-vote-number"}).string
  except AttributeError:
    votos_negativos = "?"
  
  try:
    karma = contenido.find("span", attrs = {"class":"karma-number"}).string
  except AttributeError:
    karma = "?"

  try:
    comentarios = int(contenido.find("div", attrs = {"class":"news-details-main"}).a["data-comments-number"])
  except AttributeError:
    comentarios = "?"

  try:
    enlace_hist=contenido.find("div", attrs = {"class":"votes"}).a["href"]
  except AttributeError:
    enlace_hist = "?"
  info_noticia =[codigo_noticia,titular,enlace_original,web_original,user_id,user_id_numerica,user_nick,
                 etiqueta_principal,numero_de_clics,votos_positivos,votos_anonimos,votos_negativos,karma,comentarios,enlace_hist]

  return info_noticia









#Función para buscar de forma anidada cada noticia de cada página y extraer; subetiquetas y comentarios.

Devuelve un lista que contiene dos listas: una donde cada elemento contiene las subetiquetas de cada noticia, y la otra donde cada elemento contiene los comentarios y la información de cada noticia. 

In [ ]:
def sub_historia(enlace):

  url = "https://www.meneame.net"+enlace

  page = requests.get(url, headers=headers)

  contenido = bs(page.content)

  lista_etiquetas_secundarias = []

  lista_info_comentarios = []

  lista_sub_hist = []

  #Obtenemos las etiquetas dsecudarias para la noticia

  noticias = contenido.select("html body div#variable div#wrap div#container div#newswrap div.news-summary div.news-body div.center-content")
  try:
    etiquetas_secundarias = noticias[0].find("span", attrs = {"class":"news-tags"}).find_all("a")

    lista_etiquetas_secundarias = [etiquetas_secundarias.string for etiquetas_secundarias in etiquetas_secundarias]
  
  except IndexError:

    lista_etiquetas_secundarias = "Error"


# Se capturan los comentarios y sus propiedades

  comentarios_info = contenido.select("html body div#variable div#wrap div#container")

  if len(comentarios_info)!=0:

    comentarios = comentarios_info[0].find_all("div", attrs = {"class":"comment-body"})
    comentarios_pies = comentarios_info[0].find_all("div", attrs={"class":"comment-footer"})

    for i  in range(len(comentarios)):
    
      #Necesito una lista en blanco donde poner la información de cada comentario en cada ciclo
    
      info_comentario= []
      comentario = comentarios[i]
      comentario_pie =  comentarios_pies[i]

    
      try:
        indice_comentario = comentario.find("a", attrs ={"class":"comment-order"}).string
    
        info_comentario.append(indice_comentario)
      except:
        indice_comentario = "?"
    
        info_comentario.append(indice_comentario)      

      try: 
        comentario_responde = comentario.find("div",attrs={"class":"comment-text"}).a.string 
            
        info_comentario.append(comentario_responde)

      except:

        comentario_responde = "Noticia Original"
        info_comentario.append(comentario_responde)

      try:
        siguiendo_al_autor = comentario.find("a", attrs = { "class":"comment-order"})["rel"]
        info_comentario.append(siguiendo_al_autor)
      except:
        siguiendo_al_autor = "?"
        info_comentario.append(siguiendo_al_autor)
      try:
        user_nick = comentario.find("a", attrs ={"class":"username"}).string
        info_comentario.append(user_nick)
      except:
        user_nick = "?"
        info_comentario.append(user_nick)


      try:
        user_id_numerica = comentario.find("img", attrs ={"alt":user_nick})["class"][2][2:]
        info_comentario.append(user_id_numerica)
      except:
        user_id_numerica ="?"
        info_comentario.append(user_id_numerica)

      try:
        votos = comentario_pie.find("a", attrs ={"title":"Votos"}).string
        info_comentario.append(votos)
      except:
        votos ="?"
        info_comentario.append(votos)

      try:
        karma = comentario_pie.findChild("span",attrs={"title":"Karma"}).i.next_element.next_element
        info_comentario.append(karma)
      except:
        karma ="?"
        info_comentario.append(karma)



      try:
        fecha_unix = int(comentario.find("span", attrs={"title":"creado: "})["data-ts"])

        hora_madrid_delta = 2

        anadir_hora = timedelta(hours = hora_madrid_delta)

        fecha = datetime.utcfromtimestamp(fecha_unix) + anadir_hora

        fecha = fecha.strftime('%Y-%m-%d %H:%M:%S')
        info_comentario.append(fecha)

      except:

        fecha = "?"
        info_comentario.append(fecha)

      try:
        try:
          comentario_texto = comentario.find("div",attrs={"class":"comment-text"}).a.next_element.next_element
          info_comentario.append(comentario_texto)
        except:
          if comentario.find("div", attrs= {"class":"comment-text"}).string != None:
            comentario_texto = comentario.find("div", attrs= {"class":"comment-text"}).string
            info_comentario.append(comentario_texto)
          else:
              lista_delimitadores =["<br/>", "</div>","<i>","</i>","<div class=","comment-text"," id=","cid-",">"]
              comentario_texto = str(comentario.find("div", attrs= {"class":"comment-text"}))
              for lista_delimitadores in lista_delimitadores:
                comentario_texto = comentario_texto.replace(lista_delimitadores, "")
                comentario_texto
              info_comentario.append(comentario_texto)

      except:

        comentario_texto = "?"
        info_comentario.append(comentario_texto)
     
    
      lista_info_comentarios.append(info_comentario)

  else:
    info_comentario=[]
    indice_comentario = "Error general al extraer comentario"
    info_comentario.append(indice_comentario)
    comentario_responde = "Error general al extraer comentario"
    info_comentario.append(comentario_responde)
    siguiendo_al_autor = "Error general al extraer comentario"
    info_comentario.append(siguiendo_al_autor)
    user_nick = "Error general al extraer comentario"
    info_comentario.append(user_nick)
    user_id_numerica ="Error general al extraer comentario"
    info_comentario.append(user_id_numerica)
    votos = "Error general al extraer comentario"
    info_comentario.append(votos)
    karma = "Error general al extraer comentario"
    info_comentario.append(karma)


    fecha = "Error general al extraer comentario"
    info_comentario.append(fecha)
    comentario_texto = "Error general al extraer comentario"
    info_comentario.append(comentario_texto)


    lista_info_comentarios.append(info_comentario)



  lista_sub_hist = [lista_etiquetas_secundarias,lista_info_comentarios]
  
  
  return lista_sub_hist

#Función para ordenar la información extraída de los comentarios de cada noticia en un diccionario.

In [ ]:
def extraer_lista_comentarios(lista_comentarios):

  numero_sub_cmentario = []
  referido = []
  siguiendo = []
  nick_autor_com = []
  id_numerica_autor_comen = []
  votos_comentario = []
  karma_comentario = []
  fecha_comentario = []
  cuerpo_comentario = []


  numero_sub_cmentario.extend([item[0] for item in lista_comentarios])
  referido.extend([item[1] for item in lista_comentarios])
  siguiendo.extend([item[2] for item in lista_comentarios])
  nick_autor_com.extend([item[3] for item in lista_comentarios])
  id_numerica_autor_comen.extend([item[4] for item in lista_comentarios])
  votos_comentario.extend([item[5] for item in lista_comentarios])
  karma_comentario.extend([item[6] for item in lista_comentarios])
  fecha_comentario.extend([item[7] for item in lista_comentarios])
  cuerpo_comentario.extend([item[8] for item in lista_comentarios])

  data_comentarios = {"Índice":numero_sub_cmentario,"Referido":referido,"Sige":siguiendo,
          "Nick autor comentario":nick_autor_com, "id numérica":id_numerica_autor_comen,
          "Votos":votos_comentario,"Karma":karma_comentario, "Fecha": fecha_comentario,
          "Texto":cuerpo_comentario}



  return data_comentarios









  


#Las librerías para el scrapper

In [ ]:
#Importamos las librerías necesarias

import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from numpy import random
from time import sleep
from datetime import datetime
from datetime import timedelta
from datetime import date

#EL Scrapper en sí.

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:80.0) Gecko/20100101 Firefox/80.0'}

#Listas para iterar
informacion = []
lista_envios = []
lista_publicados =[]


#Listas en las que encapsular datos

codigo_noticia =[]
titulares = []
#######
lista_fechas_envio_titulares = []
lista_fechas_publicadas = []
#######
web_original = []
user_id_numerica = []
user_nick = []
etiqueta_principal = []
numero_de_clics = []
votos_positivos = []
votos_anonimos = []
votos_negativos =[]
karma = []
comentarios = []
enlace_hist = []
sub_etiqueta_ = []

sub_etiqueta_0_1 = []
sub_etiqueta_1_1 = []
sub_etiqueta_0_1 = []
sub_etiqueta_1_1 = []
sub_etiqueta_2_1 = []
sub_etiqueta_3_1 = []
sub_etiqueta_4_1 = []

#Número de páginas sobre las que extraer información

paginas = 50


#Extracción de datos

for i in range(paginas):

  url = "https://www.meneame.net/?page="+str(i+1)
  page = requests.get(url, headers=headers)
  contenido = bs(page.content)

  noticias = contenido.select("html body div#variable div#wrap div#container div#newswrap div.news-summary")
  
  try:

    informacion = [info_noticia(noticias) for noticias in noticias]
    
    lista_envios =  [envio_de_noticia(noticias) for noticias in noticias]

    lista_publicados = [publicado_noticia(noticias) for noticias in noticias]



  except KeyError:

    informcion = str(i+1)

    lista_envios = str(i+1)

    lista_publicados = str(i+1)




#Generación de listas con datos
  
  codigo_noticia.extend([item[0] for item in informacion])
  titulares.extend([item[1] for item in informacion])
  lista_fechas_envio_titulares.extend(lista_envios)
  lista_fechas_publicadas.extend(lista_publicados)
  web_original.extend([item[3] for item in informacion])
  user_id_numerica.extend([item[5] for item in informacion])
  user_nick.extend([item[6] for item in informacion])
  etiqueta_principal.extend([item[7] for item in informacion])
  numero_de_clics.extend([item[8] for item in informacion])
  votos_positivos.extend([item[9] for item in informacion])
  votos_anonimos.extend([item[10] for item in informacion])
  votos_negativos.extend([item[11] for item in informacion])
  karma.extend([item[12] for item in informacion])
  comentarios.extend([item[13] for item in informacion])

  
#Busquedad anidada en cada historia/noticia  

  sub_etiqueta_0 = []
  sub_etiqueta_1 = []
  sub_etiqueta_2 = []
  sub_etiqueta_3 = []
  sub_etiqueta_4 = []

  for informacion in informacion:
    
    print(informacion)
    
    lista=[]
    lista_comentarios =[]
    lista,lista_comentarios =sub_historia(informacion[14])  

    info_comentarios = extraer_lista_comentarios(lista_comentarios)

    comentarios_scrap = pd.DataFrame(info_comentarios)

    comentarios_scrap.to_csv('/content/drive/My Drive/Colab Notebooks/info_meneame/comentarios_noticia_+'+str(informacion[0])+str(date.today())+'.csv')

    try:
      sub_etiqueta_0.append(lista[0])
    except IndexError:
      sub_etiqueta_0.append("?_?")
    try:
      sub_etiqueta_1.append(lista[1])
    except IndexError:
      sub_etiqueta_1.append("?_?")
    try:
      sub_etiqueta_2.append(lista[2])
    except IndexError:
      sub_etiqueta_2.append("?_?")
    try:
      sub_etiqueta_3.append(lista[3])
    except IndexError:
      sub_etiqueta_3.append("?_?")
    try:
      sub_etiqueta_4.append(lista[4])
    except IndexError:
      sub_etiqueta_4.append("?_?")




    sleep(random.uniform(0.5, 1))

    


  sub_etiqueta_0_1.extend(sub_etiqueta_0)
  sub_etiqueta_1_1.extend(sub_etiqueta_1)
  sub_etiqueta_2_1.extend(sub_etiqueta_2)
  sub_etiqueta_3_1.extend(sub_etiqueta_3)
  sub_etiqueta_4_1.extend(sub_etiqueta_4)

#tiempos de sleep para no saturar el servidor 

  sleep(random.uniform(2, 4))

#Comprobación de la url sobre la que se ha extraído información

  print(url)

#Salvamos los datos generales


data = {"código":codigo_noticia,"Titular": titulares, "Fecha de envío": lista_fechas_envio_titulares,
        "Publicada": lista_fechas_publicadas,"Web origen": web_original,"codigo usuario":user_id_numerica,
        "Nick":user_nick,"Tema general":etiqueta_principal,"Sub Tema 1":sub_etiqueta_0_1,"Sub Tema 2":sub_etiqueta_1_1,
        "Sub Tema 3":sub_etiqueta_2_1,"Sub Tema 4":sub_etiqueta_3_1,"Sub Tema 5":sub_etiqueta_4_1,
        "Clicks":numero_de_clics, "Positivos": votos_positivos,
        "Anónimos":votos_anonimos,"Negativos":votos_negativos, "Karma":karma, "Número de comentarios":comentarios}

df_scrap = pd.DataFrame(data)

df_scrap.to_csv('/content/drive/My Drive/Colab Notebooks/info_meneame/meneame'+str(date.today())+'.csv',index=False )



[3388214, ' Las provincias españolas con el mejor sueldo medio: hasta 26.000 euros al año ', 'https://www.lainformacion.com/management/mejores-sueldos-provincias-empleos/2816632/', 'lainformacion.com', 'u:625099', '625099', 'Condetino', 'actualidad', '454', '17', '5', '2', '344', 18, '/story/provincias-espanolas-mejor-sueldo-medio-hasta-26-000-euros-ano']
[3388279, ' Villarejo asegura que comía con el juez Manuel Marchena una vez al mes en la Sierra de Madrid ', 'https://www.elconfidencialdigital.com/articulo/Judicial/villarejo-asegura-comia-juez-manuel-marchena-vez-mes-sierra-madrid/20201001173103168665.html', 'elconfidencialdigital.com', 'u:375825', '375825', 'candonga1', 'actualidad', '408', '129', '97', '6', '321', 38, '/story/villarejo-asegura-comia-juez-manuel-marchena-vez-mes-sierra']
[3388091, ' El farero que tragó plomo fundido y sobrevivió (unos días) para contarlo ', 'https://www.larazon.es/ciencia/20201001/s5h6xftsynelnpny4m56jeunhy.html', 'larazon.es', 'u:95524', '95524', 

In [ ]:
df_scrap.head()

,código,Titular,Fecha de envío,Publicada,Web origen,codigo usuario,Nick,Tema general,Sub Tema 1,Sub Tema 2,Sub Tema 3,Sub Tema 4,Sub Tema 5,Clicks,Positivos,Anónimos,Negativos,Karma,Número de comentarios
0,3388214,Las provincias españolas con el mejor sueldo ...,2020-10-02 08:26:31,2020-10-02 13:30:03,lainformacion.com,625099,Condetino,actualidad,provincias,españolas,mejor,sueldo,medio,454,17,5,2,344,18
1,3388279,Villarejo asegura que comía con el juez Manue...,2020-10-02 09:52:46,2020-10-02 12:50:25,elconfidencialdigital.com,375825,candonga1,actualidad,villarejo,juez marchena,sierra de madrid,?_?,?_?,408,129,97,6,321,38
2,3388091,El farero que tragó plomo fundido y sobrevivi...,2020-10-01 22:20:18,2020-10-02 12:05:23,larazon.es,95524,painful,ciencia,farero,plomo,faro de eddington,herny hall,?_?,3849,46,26,8,319,38
3,3388219,Mamen Mendizábal pregunta a Pablo Montesinos ...,2020-10-02 08:40:42,2020-10-02 11:20:09,twitter.com,598828,Stiller,actualidad,mamen mendizábal,pablo montesino,cam. covid,medidas,?_?,3847,205,251,35,342,98
4,3388190,PSOE y PP acuerdan mantener en 60 años la jub...,2020-10-02 07:22:02,2020-10-02 10:40:35,eleconomista.es,660999,Fraipericoysuborrico,actualidad,jubilación,funcionarios,mantener,pp,psoe,1831,115,143,19,334,262
